# ENSF 612 Project

Justin Nguyen  - 30042258 <br>
Garnet Crookes - 30173480<br>
Ben Kaminski   - 00301335<br>

# 0. Setup

In [1]:
%pip install bs4
%pip install xlsxwriter
%pip install nltk

%pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
import nltk
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\bdict\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [31]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bdict\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bdict\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# 1. Get Reviews and AppID

In [141]:
def getReviews(appId, n=100):
    reviews = []
    cursor = '*'
    params = {
            'json' : 1,
            'filter' : 'all',
            'language' : 'english',
            'day_range' : 9223372036854775807,
            'review_type' : 'all',
            'purchase_type' : 'all'
            }

    while n > 0:
        params['cursor'] = cursor.encode()
        params['num_per_page'] = min(100, n)
        n -= 100

        url = 'https://store.steampowered.com/appreviews/'
        response = requests.get(url=url+appId, params=params, headers={'User-Agent': 'Mozilla/5.0'}).json()
        cursor = response['cursor']
        reviews += response['reviews']

        if len(response['reviews']) < 100: break

    return reviews

def getAppId(game_name):
    response = requests.get(url=f'https://store.steampowered.com/search/?term={game_name}&category1=998', 
                            headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(response.text, 'html.parser')
    app_id = soup.find(class_='search_result_row')['data-ds-appid']
    return app_id

In [142]:
getReviews(getAppId('FIFA 23'))

[{'recommendationid': '123102831',
  'author': {'steamid': '76561198030783130',
   'num_games_owned': 10,
   'num_reviews': 1,
   'playtime_forever': 8141,
   'playtime_last_two_weeks': 3351,
   'playtime_at_review': 7,
   'last_played': 1669956623},
  'language': 'english',
  'review': "I guess the anticheat is working, because if you can't play the game you can't cheat.",
  'timestamp_created': 1664575412,
  'timestamp_updated': 1664575412,
  'voted_up': False,
  'votes_up': 1057,
  'votes_funny': 737,
  'weighted_vote_score': '0.962865948677062988',
  'comment_count': 0,
  'steam_purchase': True,
  'received_for_free': False,
  'written_during_early_access': False,
  'hidden_in_steam_china': True,
  'steam_china_location': ''},
 {'recommendationid': '125623203',
  'author': {'steamid': '76561198845938257',
   'num_games_owned': 202,
   'num_reviews': 24,
   'playtime_forever': 508,
   'playtime_last_two_weeks': 359,
   'playtime_at_review': 95,
   'last_played': 1670094700},
  'lang

# 2. Export Dataframes for Labeling

In [143]:
import pandas as pd

users = {"Justin":['FIFA 23', 'New World', 'For Honor'], 
		"Ben" : ['PUBG', 'Project CARS 3', 'Path of Exile'], 
		"Garnet": ['Call of Duty Modern Warfare II', 'Harvest Moon: Light of Hope', 'Total War: Warhammer']}

for user in users:
	with pd.ExcelWriter(user + ".xlsx", engine = 'xlsxwriter') as writer:
		for game in users[user]:
			df = pd.DataFrame(getReviews(getAppId(game),400))[['review', 'voted_up', 'weighted_vote_score', 
                                                                'author', "recommendationid"]]
			df.to_excel(writer, sheet_name = game.replace(":",""), index = False)

# 3. Import Labeled Excel Files

In [158]:
files = ["Justin.xlsx", "Ben.xlsx", "Garnet.xlsx", "BeatSaber.xlsx"]

df = pd.DataFrame()

for file in files:

	sheets = pd.read_excel(file, sheet_name=None)

	for name in sheets.keys(): 
		df_temp = sheets[name]
		df_temp["game_name"] = name
		df = pd.concat([df, df_temp],ignore_index= True)


df["author_dict"] = df["author"].apply(lambda x: eval(x))
df = pd.concat([df, df["author_dict"].apply(pd.Series)], axis= 1)
df = df.drop(["author", "author_dict"], axis = 1)

In [159]:
df[df.index.duplicated()]
df

,label,review,voted_up,weighted_vote_score,recommendationid,game_name,steamid,num_games_owned,num_reviews,playtime_forever,playtime_last_two_weeks,playtime_at_review,last_played
0,nonconstructive,"I guess the anticheat is working, because if y...",False,0.962647,123102831,FIFA 23,76561198030783130,9,1,3684,874,7.0,1668480418
1,nonconstructive,EA never disappoint us when it comes to disapp...,False,0.954147,123032503,FIFA 23,76561198142247081,541,144,397,0,10.0,1664897993
2,nonconstructive,I know who made the game. I bought this game k...,False,0.952997,123113040,FIFA 23,76561198070975404,288,76,3429,0,581.0,1665197381
3,constructive,Pathetic. Utterly pathetic. \n\nI'm so glad I ...,False,0.945785,123258303,FIFA 23,76561198079724542,156,7,397,0,379.0,1666653313
4,nonconstructive,if you are buying this then you have only two ...,False,0.945166,124259509,FIFA 23,76561198275239160,61,9,27,0,27.0,1664624403
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1991,Nonconstructive,the most fun i've had in vr,True,0.659699,118688268,Beat Saber,76561198184713498,84,14,253,0,253.0,1655073174
1992,nonconstructive,YES,True,0.658929,74201960,Beat Saber,76561198200491171,301,14,4165,0,904.0,1657109988
1993,Constructive,"While I enjoy rhythm based games, they are not...",True,0.658746,49348139,Beat Saber,76561198017663146,1673,118,573,0,307.0,1585885588
1994,Constructive,"Fun, intensive rhythm game. You either love it...",True,0.658692,42144229,Beat Saber,76561198002634802,347,7,38215,0,1247.0,1656286594


# 4. Preprocessing

In [160]:
lemmatizer = WordNetLemmatizer()

def preprocessing(review):
    # remove stopwords
    new_stopwords = ["harvest", "moon", "light", "call", "duty", "modern", "warfare", "cars", 
                     "project", "total", "warhammer", "path", "exile", "sims", "game", "unknown", 
                     "pubg", "players", "shooter"]
    stop_words = set(stopwords.words('english') + new_stopwords)
    words = word_tokenize(review.lower()) 
    sw_filter = ""
    for w in words:
        if w not in stop_words:
            sw_filter += w + " "
            
    # remove punctuation marks
    words = word_tokenize(sw_filter) 
    punc_filter = ""
    for w in words:
        punc_filter += ''.join([i for i in w if i.isalnum()]) + " "
        
    # remove numbers
    words = word_tokenize(punc_filter) 
    num_filter = ""
    for w in words:
        num_filter += ''.join([i for i in w if not i.isdigit()]) + " "
        
    # remove small/incomplete words
    words = word_tokenize(num_filter)
    small_filter = ""
    for w in words:
        if len(w) > 3:
            small_filter += ''.join([i for i in w]) + " "
    
    # lemmatizer
    lemmatizer = WordNetLemmatizer()
    words = word_tokenize(small_filter) 
    lem_filter = ""
    for w in words:
        lem_filter += lemmatizer.lemmatize(w) + " "

    return lem_filter

df['review_processed'] = df['review'].apply(lambda x: preprocessing(str(x)))

In [161]:
df = pd.DataFrame(df)
df['review_processed'].astype(pd.StringDtype())

0                     guess anticheat working play cheat 
1                    never disappoint come disappointing 
2             know made bought knowing made brought upon 
3       pathetic utterly pathetic glad played eaplay s...
4                                     buying hour refund 
                              ...                        
1991                                                     
1992                                                     
1993    enjoy rhythm based game normal waited beat sab...
1994    intensive rhythm either love type dance dance ...
1995                          extremely even amazing mod 
Name: review_processed, Length: 1996, dtype: string

In [162]:
print(df.loc[[59]])

              label                                       review  voted_up  \
59  nonconstructive  Why EA anticheat? failed always.... so baad     False   

    weighted_vote_score  recommendationid game_name            steamid  \
59             0.732015         123016247   FIFA 23  76561198134729795   

    num_games_owned  num_reviews  playtime_forever  playtime_last_two_weeks  \
59               38            4               831                        0   

    playtime_at_review  last_played               review_processed  
59                 8.0   1665039003  anticheat failed always baad   


In [163]:
df=df['review_processed'].replace("",np.nan)
df.dropna(subset=['review_processed'])



,label,review,voted_up,weighted_vote_score,recommendationid,game_name,steamid,num_games_owned,num_reviews,playtime_forever,playtime_last_two_weeks,playtime_at_review,last_played,review_processed
0,nonconstructive,"I guess the anticheat is working, because if y...",False,0.962647,123102831,FIFA 23,76561198030783130,9,1,3684,874,7.0,1668480418,guess anticheat working play cheat
1,nonconstructive,EA never disappoint us when it comes to disapp...,False,0.954147,123032503,FIFA 23,76561198142247081,541,144,397,0,10.0,1664897993,never disappoint come disappointing
2,nonconstructive,I know who made the game. I bought this game k...,False,0.952997,123113040,FIFA 23,76561198070975404,288,76,3429,0,581.0,1665197381,know made bought knowing made brought upon
3,constructive,Pathetic. Utterly pathetic. \n\nI'm so glad I ...,False,0.945785,123258303,FIFA 23,76561198079724542,156,7,397,0,379.0,1666653313,pathetic utterly pathetic glad played eaplay s...
4,nonconstructive,if you are buying this then you have only two ...,False,0.945166,124259509,FIFA 23,76561198275239160,61,9,27,0,27.0,1664624403,buying hour refund
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1991,Nonconstructive,the most fun i've had in vr,True,0.659699,118688268,Beat Saber,76561198184713498,84,14,253,0,253.0,1655073174,
1992,nonconstructive,YES,True,0.658929,74201960,Beat Saber,76561198200491171,301,14,4165,0,904.0,1657109988,
1993,Constructive,"While I enjoy rhythm based games, they are not...",True,0.658746,49348139,Beat Saber,76561198017663146,1673,118,573,0,307.0,1585885588,enjoy rhythm based game normal waited beat sab...
1994,Constructive,"Fun, intensive rhythm game. You either love it...",True,0.658692,42144229,Beat Saber,76561198002634802,347,7,38215,0,1247.0,1656286594,intensive rhythm either love type dance dance ...


In [164]:
with pd.ExcelWriter("fullwithlemonlydropempty.xlsx", engine = 'xlsxwriter') as writer:
     df.to_excel(writer, sheet_name = "total", index = False)